# Osnovi računarske inteligencije

## Vežba 3 - Pretrage sa suparnikom/protivnikom

---

Igre su oduvek bile uzrok (velikih) intelektualnih napora ljudi i samim tim su od začetka oblasti veštačke (računarske) inteligencije veoma interesantan problem. Da li mašina koja u nekoj igri može da pobedi čoveka ima inteligenciju? Da li to znači da je "pametnija" od čoveka?

---

### Igre

Igranje igre - rešavanje problema (pretragama u našem slučaju) u kompetitivnom i često nepredvidljivom okruženju sa više agenata.

Jednostavne igre:
* Dva igrača
* Igraju potezno
* Savršene informacije - sve potpuno vidiljivo
* Bez nasumičnih elemenata - determinističke
* Mali broj mogućih akcija
* Precizna, formalna pravila
* Ne zahtevaju puno znanja (na prvi pogled)

Igre su interesantne za AI jer:
* Uglavnom su preteške da bi se optimalno rešile
* Uglavnom se neefikasnost kažnjava (pretraga mora biti brza)
* Neke imaju nasumične elemente (npr. kocka)
* Neke imaju nesavršene informacije (npr. ne vide se protivnikove karte)

---

### Igre kao problem pretrage

Igra je opisana:
1. Inicijalnim stanjem - konfiguracija i koji je igrač na potezu
2. Funkcijom koja generiše naredna stanja iz inicijalnog stanja
3. Terminalni testom - da li je igra gotova i ko je pobednik, ako ga ima
4. Evaluacionom funkcijom - mapira stanja na numeričke vrednosti (na osnovu čega se procenjuje koliko je svako stanje "dobro").

Postoje dva igrača: tzv. **MAX** i **MIN** igrači. MAX igrač želi da maksimizuje vrednost evaluacione funkcije, dok MIN igrač želi da je minimizuje. MAX pretpostavlja se da protivnik (MIN) ne greši, i MIN pretpostavlja isto za MAX igrača.

---

### Minimax

Razvijanje stabla igre prvi u dubinu, do predefinisane dubine. Na krajnjim čvorovima/stanjima se računa numerička vrednost na osnovu evaluacione funkcije. MIN igrač će uvek birati putanju sa minimalnom vrednošću evaluacione funkcije, dok će MAX igrač uvek birati putanju koja vodi ka maksimalnoj vrednosti.

![minimax.gif](img/minimax.gif)

Pseudo-kod (sa Wikipedia stranice):

```
function minimax(node, depth, maximizingPlayer)
     if depth = 0 or node is a terminal node
         return the heuristic value of node

     if maximizingPlayer
         bestValue := −∞
         for each child of node
             v := minimax(child, depth − 1, FALSE)
             bestValue := max(bestValue, v)
         return bestValue

     else    (* minimizing player *)
         bestValue := +∞
         for each child of node
             v := minimax(child, depth − 1, TRUE)
             bestValue := min(bestValue, v)
         return bestValue
         
(* Initial call for maximizing player *)
minimax(origin, depth, TRUE)
```

* Ako je faktor grananja stabla igre **b** i ako je dubina pretrage **d**, za koliko čvorova/stanja se računa vrednost evaluacione funkcije? Ovo ćemo nazivati *kompleksnost* i u ovom slučaju je $ O(b^d) $
* Po vašoj pretpostavci, koliko su **b** i **d** u šahu (ako se simulira cela partija)?

**TODO 1**: Implementirati minimax algoritam pretrage. ```search.py``` -> ```Minimax```.

---

### Alpha-beta

Alpha-beta pruning (odsecanje) je proširenje Minimax algoritma, koje smanjuje broj čvorova/stanja za koje se računa evaluaciona funkcija, što znatno smanjuje potrebnu računsku moć. Neki potezi (i samim tim svi oni koji slede iz njih) *nikad neće biti razmatrani*, odnosno biće *odsečeni*. Alpha-beta algoritam vraća potpuno isti rezultat kao i Minimax algoritam, ali odseca grane stabla koje ne mogu uticati na konačnu odluku.

![alphabeta.gif](img/alphabeta.gif)

Pseudo-kod (sa Wikipedia stranice):

```
function alphabeta(node, depth, α, β, maximizingPlayer)
      if depth = 0 or node is a terminal node
          return the heuristic value of node
      if maximizingPlayer
          v := -∞
          for each child of node
              v := max(v, alphabeta(child, depth - 1, α, β, FALSE))
              α := max(α, v)
              if β ≤ α
                  break (* β cut-off *)
          return v
      else
          v := ∞
          for each child of node
              v := min(v, alphabeta(child, depth - 1, α, β, TRUE))
              β := min(β, v)
              if β ≤ α
                  break (* α cut-off *)
          return v
          
(* Initial call *)
alphabeta(origin, depth, -∞, +∞, TRUE)
```

U idealnom slučaju, alpha-beta smanjenjuje kompleksnost na $ O(b^{d/2}) $, što praktično omogućava *duplo veću dubinu pretrage za istu količinu resursa (vreme)*. Idealni slučaj je kada su naredna stanja (potezi) sortirana od najboljeg ka najgorem, tako da se odsecanje dešava što je ranije moguće. U slučaju kada naredna stanja nisu sortirana, alpha-beta i dalje smanjuje kompleksnost na *otprilike* $ O(b^{3d/4}) $. Ipak, ovo i dalje nije dovoljno smanjilo kompleksnost da može se reši cela partija šaha.


**TODO 2**: Implentirati alpha-beta algoritam pretrage. ```search.py``` -> ```AlphaBeta```.

---


### Evaluacione funkcije

Za svako stanje vraća numeričku vrednost koliko je neko stanje "dobro". Dobra evaluaciona funkcija bi trebala da se izvršava brzo, i da oslikava realno stanje u igri (iako je to samo aproksimacija).

Uglavnom je linearna kombinacija nekih informativnih osobina stanja:

$$ e(s) = w_1f_1(s) + w_2f_2(s) + ... + w_nf_n(s) $$

*Problem*: otkriti **težine ($ w_i $)** za koje će AI dobro raditi/igrati.

Primeri informativnih osobina stanja (u šahu):
* Brojčano stanje figura
* Struktura pijuna
* Bezbednost kralja
* Kontrola sredine table
* Mobilnost (više raspoloživih poteza je bolje)

U praksi:
* Težine se mogu menjati tokom partije
* Kombinacije informativnih osobina ne moraju biti linearne

Kako biramo težine i osobine za evaluacionu funkciju?
* Iskustvo ljudi u samoj igri
* Mašinsko učenje (vrednosti se poboljšavaju tokom vremena, često tako što se stave dve AI jedne protiv druge, iznova i iznova)

---

#### Prostor za napredak

* Sortiranje poteza od najboljeg ka najgorem
* Pamtiti dobre poteze od pre pa ih koristiti u narednim partijama
* Za šah je korisno imati "knjigu otvaranja"
* Iterativno produbljivanje - ukoliko je igra vremenski ograničena, možda u nekim situacijama nije moguće istražiti sve poteze. Stoga se iterativno pronalazi potez do neke male dubine, pa ponovo do malo veće, itd. Tako smo sigurni da imamo spreman potez (kakav god) u bilo kom trenutku - u nekim igrama kada istekne vreme i nemate potez izgubili ste. (Iterativno produbljivanje takođe se može iskoristiti za sortiranje poteza u sledećoj iteraciji).
* Singularne ekstenzije - za poteze koji su očigledno bolji od ostalih, produbiti stablo kako bi se dodatno istražili
* Dubina pretrage - tzv. horizont događaja, posle te dubine više ne znamo šta se događa. Kolika god da je dubina *d*, uvek u *d+1* može biti napravljen neki loš potez

#### Drugi algoritmi (varijacije minimax i alpha-beta)

* Negascout - varijacija alpha-beta, koja daje poboljšanje u odnosu na alpha-beta od oko 10%, ali samo ako se radi sortiranje poteza.
* MTD(f) - teoretski bolji i efikasniji od Negascout, iako postoje određene praktični problemi
* SSS\* - radi razvoj stabla prvi u širinu, a ne prvi u dubinu (što može dovesti do problema sa memorijom)


#### Dodatni zadaci

* Razne optimizacije - trenutno ovo je prilično "školski" primer, što znači da je i prilično neefikasan/spor/memorijski zahtevan (dobar resurs za ideje https://chessprogramming.wikispaces.com/, poglavlja Board representation, Search, Evaluation)
* Bolja evaluaciona funkcija
* Razna poboljšanja - pogledati gore *Prostor za napredak*

---

### Kratka istorija AI u šahu

Rangiranje u šahu:
* 500+ - loš igrač, zna samo pravila
* 1200+ - "casual" igrač, igra šah već neko vreme
* 2000+ - svetski rangiran igrač, majstor
* 2500+ - velemajstor
* 2800+ - Kasparov, Fišer, Karlsen...

 
1. Ranih 1950ih Shannon i Turing napravili program koji može da igra osnovi šah (jedva). Rank 500.
2. Sredinom 1950ih Alex Bernstein napravio manja poboljšanja. Rank 500 (+ nešto malo).
3. 1957\. Herb Simon tvrdi da će računarski šahovski program biti svetski šampion u šahu... yeah, right.
4. 1966\. McCarthy organizuje šahovski meč između Stanforda i Rusije. Dugačak meč. Rusija pobeđuje.
5. 1967\. Richard Greenblatt sa MIT napravio prvi moderni program koji igra šah. Rank 1100.
6. 1968\. McCarthy, Michie i Papert se klade protiv Levy-ja (rank 2325) da će ga računar pobediti u narednih 10 godina.
7. 1970\. ACM (Association for Computing Machinery) počeo da organizuje šahovske turnire za računare. Pobednik Chess 3.0-3.6, rank 1400.
8. 1973\... Slate: “It had become too painful even to look at Chess 3.6 any more, let alone work on it.”
9. 1973\. Chess 4.0: pametniji generator poteza - znatno ubrzanje na bržim računarima.
10. 1976\. Chess 4.5: rank 2070.
11. 1977\. Chess 4.5 protiv Levy. Levy pobeđuje.
12. 1980ih godina - programi zavise najviše od brzine pretrage, a ne od znanje. Rank do 2300.
13. 1993\. DEEP THOUGHT - sofisticirani računar za specijalne svrhe; pretrage; 10 poteza unapred; singularne ekstenzije. Rank ~2600.
14. 1995\. DEEP BLUE (IBM) - 14 poteza unapred; iterativno produbljivanje; razmatra 100-200 milijardi pozicija po potezu; evaluacioni funkcija koja ima preko 8000 osobina; singularne ekstenzije do 40 poteza unapred; knjiga otvaranja sa 4000 pozicija; knjiga završnice (kada je na tabli samo 5-6 figura). Naravno sve ovo na specijalno napravljanom računaru.
15. 1997\. DEEP BLUE protiv Kasparova. DEEP BLUE pobeđuje.
16. 2002\. IBM odbija revanš protiv Kasparova.
17. 2002\. FRITZ protiv svetskog šampiona Vladimira Kramnika. 8 partija. Nerešeno.
18. 2002. - 2017. Računarski algoritmi napreduju, ali u osnovi ostaju isti. Računari daleko nadjačavaju najbolje ljudske igrače. Osnivaju se takmičenja u igranju šaha za računare.
19. 2017. Google DeepMind izbacuje AlphaZero - agent zasnovan na Deep Reinforcement Learning-u koji je sposoban da nauči igru igrajući sam protiv sebe. Pobeđuje tada najboljeg **chess engine - Stockfish** u meču od 100 partija sa rezultatom  
28-72-0.  Ovaj događaj započinje revoluciju u svetu šaha, gde trenutno svi najuspešniji algoritmi koriste neuronske mreže u nekom obliku. (Stockfish, Leela Chess Zero, Komodo)
